In [1]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
import pandas as pd


In [2]:
csv_path = "data/salesdata_raw.csv"
sales_raw = pd.read_csv(csv_path)
sales_raw.head()

,Sell-to Customer No_,yr,mo,FiscalYear,FiscalQuarter,No_,Posting Group,units,sales,CustomerType,Description,Your Item Type Field!
0,3XL001,2014,1,2014,Q4,800501-0208,FG SEED KI,300,"$2,100.00",Retail,"SK, 7-Pod, Cherry Tomato",0.0
1,3XL001,2014,1,2014,Q4,800528-0208,FG SEED KI,100,$700.00,Retail,"SK, 7 Pod, Grow Anything",0.0
2,3XL001,2014,1,2014,Q4,800544-0208,FG SEED KI,300,"$2,100.00",Retail,"SK, 7- Pod, Chili Pepper",0.0
3,3XL001,2014,1,2014,Q4,970133-0100,ACCESSORY,30,$465.00,Retail,"Acc, AeroVoir with Stand",0.0
4,3XL001,2014,5,2015,Q1,800500-0208,FG SEED KI,500,"$3,500.00",Retail,"SK, 7- Pod, Gourmet Herb",0.0


In [3]:
#Since this dataset was from Emma's work I took direction from her on what columns were redundant.  We didn't need
#fiscal year or item type field so dropped those to start.
sales_clean = sales_raw.drop(columns=["Your Item Type Field!", "FiscalYear"])
#sales_clean.head()

In [4]:
#The column names are messy so next I cleaned those.
sales_clean.columns = sales_clean.columns.str.strip().str.lower().str.replace('_','')\
.str.replace(' ','').str.replace('-','')

#now rename the columns so they will be consistent across all three tables
sales_clean.rename(columns = {'selltocustomerno':'customerno','no':'itemno'}, inplace=True)
sales_clean.head()

,customerno,yr,mo,fiscalquarter,itemno,postinggroup,units,sales,customertype,description
0,3XL001,2014,1,Q4,800501-0208,FG SEED KI,300,"$2,100.00",Retail,"SK, 7-Pod, Cherry Tomato"
1,3XL001,2014,1,Q4,800528-0208,FG SEED KI,100,$700.00,Retail,"SK, 7 Pod, Grow Anything"
2,3XL001,2014,1,Q4,800544-0208,FG SEED KI,300,"$2,100.00",Retail,"SK, 7- Pod, Chili Pepper"
3,3XL001,2014,1,Q4,970133-0100,ACCESSORY,30,$465.00,Retail,"Acc, AeroVoir with Stand"
4,3XL001,2014,5,Q1,800500-0208,FG SEED KI,500,"$3,500.00",Retail,"SK, 7- Pod, Gourmet Herb"


In [5]:
#check the data types for the columns
sales_clean.dtypes


customerno       object
yr                int64
mo                int64
fiscalquarter    object
itemno           object
postinggroup     object
units            object
sales            object
customertype     object
description      object
dtype: object

In [6]:
#Units and sales columns are objects and we want them as float(64) for data manipulation
#Next step is to remove all characters that would prevent that conversion
#Clean the units
sales_clean['units'] = sales_clean['units'].astype(str).str.strip().str.lower().str.replace('_','')\
.str.replace(' ','').str.replace('-','').str.replace('(','').str.replace(')','').str.replace(',','')\
.str.replace('$','').str.replace('.','').str.replace('nan','')

#Clean the sales column
sales_clean['sales'] = sales_clean['sales'].astype(str).str.strip().str.lower().str.replace('_','')\
.str.replace(' ','').str.replace('-','').str.replace('(','').str.replace(')','').str.replace(',','')\
.str.replace('$','').str.replace('.','').str.replace('nan','')

#try conversion of units and sales to numeric
sales_clean['units'] = pd.to_numeric(sales_clean['units'])

sales_clean['sales'] = pd.to_numeric(sales_clean['sales'])

In [7]:
#check new dtypes
sales_clean.dtypes

customerno        object
yr                 int64
mo                 int64
fiscalquarter     object
itemno            object
postinggroup      object
units            float64
sales            float64
customertype      object
description       object
dtype: object

In [8]:
#clean the postinggroup and lowercase everything
sales_clean['postinggroup']=sales_clean['postinggroup'].str.strip().str.replace(' ', '').str.lower()
print(sales_clean['postinggroup'].unique())

['fgseedki' 'accessory' 'fggardens' 'raw' 'displays' 'seedpod' 'component'
 'ag3garden']


In [9]:
#The description column was the most messy so required quite a few steps.  First basic cleaning.
sales_clean['description'] = sales_clean['description'].astype(str).str.strip().str.lower().str.replace('_','')\
.str.replace(' ','').str.replace('-','').str.replace('(','').str.replace(')','')\
.str.replace('$','').str.replace('.','').str.replace('nan','').str.replace("'",'').str.replace('&', '')\
.str.replace('/', '').str.replace("''", "")
sales_clean.head()

,customerno,yr,mo,fiscalquarter,itemno,postinggroup,units,sales,customertype,description
0,3XL001,2014,1,Q4,800501-0208,fgseedki,300.0,210000.0,Retail,"sk,7pod,cherrytomato"
1,3XL001,2014,1,Q4,800528-0208,fgseedki,100.0,70000.0,Retail,"sk,7pod,growanything"
2,3XL001,2014,1,Q4,800544-0208,fgseedki,300.0,210000.0,Retail,"sk,7pod,chilipepper"
3,3XL001,2014,1,Q4,970133-0100,accessory,30.0,46500.0,Retail,"acc,aerovoirwithstand"
4,3XL001,2014,5,Q1,800500-0208,fgseedki,500.0,350000.0,Retail,"sk,7pod,gourmetherb"


In [10]:
#Some of this data cleaning required knowledge of what was erronious information, since this is Emma's
#company I took direction from her for this portion.
sales_clean['description'] = sales_clean['description'].str.replace('pod', 'p')\
    .str.replace('sk', '').str.replace('acc', '').str.replace('rp', '')\
    .str.replace('pack', 'pk').str.replace('w/', '').str.replace('us', '')\
    .str.replace('eu', '').str.replace('uni', '').str.replace('dlx', '')\
    .str.replace('seedstartingsystem', 'sss').str.replace('seedstartingsys', 'sss')\
    .str.replace('aggrowbowlv2growmedia', 'growbowlandgrowmedia').str.replace('ag', '')\
    .str.replace('display', '').str.replace('seedp', '').str.replace('std', '')\
    .str.replace('pop', '').str.replace('aerogdn', '').str.replace('aerogarden', '')\
    .str.replace('pdq', '').str.replace('versal', '').str.replace('bat', '').str.replace('deluxe', '')\
    .str.replace('wb', '').str.replace('gherb', 'gh').str.replace('gourmetherb', 'gh')
#Replace commas with spaces and then strip so no extra commas remain
sales_clean["description"] = sales_clean["description"].str.replace(',', ' ').str.strip()
sales_clean.head()

,customerno,yr,mo,fiscalquarter,itemno,postinggroup,units,sales,customertype,description
0,3XL001,2014,1,Q4,800501-0208,fgseedki,300.0,210000.0,Retail,7p cherrytomato
1,3XL001,2014,1,Q4,800528-0208,fgseedki,100.0,70000.0,Retail,7p growanything
2,3XL001,2014,1,Q4,800544-0208,fgseedki,300.0,210000.0,Retail,7p chilipepper
3,3XL001,2014,1,Q4,970133-0100,accessory,30.0,46500.0,Retail,aerovoirwithstand
4,3XL001,2014,5,Q1,800500-0208,fgseedki,500.0,350000.0,Retail,7p gh


In [11]:
#now we want to split the description column into new columns split along the spaces where there used to be columns
exp_sales_clean= sales_clean['description'].str.split(' ',n=-1,expand=True)
exp_sales_clean.head()

,0,1,2,3,4
0,7p,cherrytomato,None,None,None
1,7p,growanything,None,None,None
2,7p,chilipepper,None,None,None
3,aerovoirwithstand,None,None,None,None
4,7p,gh,None,None,None


In [12]:
#now I want to stitch the new df columns back into the sales data and drop the description columns
#the 5th column wasn't necessary so we didn't put it back in the df
sales_clean['desc1'] = exp_sales_clean[0]
sales_clean['desc2'] = exp_sales_clean[1]
sales_clean['desc3'] = exp_sales_clean[2]
sales_clean['desc4'] = exp_sales_clean[3]
sales_clean = sales_clean.drop(columns=["description"])
sales_clean.head()

,customerno,yr,mo,fiscalquarter,itemno,postinggroup,units,sales,customertype,desc1,desc2,desc3,desc4
0,3XL001,2014,1,Q4,800501-0208,fgseedki,300.0,210000.0,Retail,7p,cherrytomato,None,None
1,3XL001,2014,1,Q4,800528-0208,fgseedki,100.0,70000.0,Retail,7p,growanything,None,None
2,3XL001,2014,1,Q4,800544-0208,fgseedki,300.0,210000.0,Retail,7p,chilipepper,None,None
3,3XL001,2014,1,Q4,970133-0100,accessory,30.0,46500.0,Retail,aerovoirwithstand,None,None,None
4,3XL001,2014,5,Q1,800500-0208,fgseedki,500.0,350000.0,Retail,7p,gh,None,None


In [13]:
sales_clean['desc1'] = sales_clean['desc1'].str.replace('7','7p').str.replace('6','6p')\
.str.replace('3','3p').str.replace('None','').str.replace('7ppp','7p').str.replace('7pp','7p')\
.str.replace('6pp','6p').str.replace('3pp','3p')
sales_clean.fillna(value="-",inplace=True)
sales_clean.head()

,customerno,yr,mo,fiscalquarter,itemno,postinggroup,units,sales,customertype,desc1,desc2,desc3,desc4
0,3XL001,2014,1,Q4,800501-0208,fgseedki,300,210000,Retail,7p,cherrytomato,-,-
1,3XL001,2014,1,Q4,800528-0208,fgseedki,100,70000,Retail,7p,growanything,-,-
2,3XL001,2014,1,Q4,800544-0208,fgseedki,300,210000,Retail,7p,chilipepper,-,-
3,3XL001,2014,1,Q4,970133-0100,accessory,30,46500,Retail,aerovoirwithstand,-,-,-
4,3XL001,2014,5,Q1,800500-0208,fgseedki,500,350000,Retail,7p,gh,-,-


In [14]:
sales_clean.to_csv('data/sales_clean.csv', index=False) 

In [15]:
## 2) Clean 2nd Data Frame with item reference data

In [16]:
csv_path = "data/itemlookup_raw.csv"
item_raw = pd.read_csv(csv_path)
item_raw.head()

,ItemNumber,Posting Group,Description,Add Your Item Type Field Here!
0,0020-00Z,FG SEED KI,"SK, 14 UnivPods, Salad Lover's",NaN
1,0021-00Z,FG SEED KI,"SK, 14P, Herb Lover's Seed Kit",NaN
2,100235-0000,RAW,Flat Plant Spacer (China),NaN
3,100824-0000,RAW,SuperGrow Nutrients (3oz.),NaN
4,100828-0000,ACCESSORY,Aerovoir Stand,NaN


In [17]:
#Drop the Add your item type field here column
item_clean = item_raw.drop(columns=['Add Your Item Type Field Here!'])

In [18]:
#Clean the column titles and rename ItenNumber to itemno
item_clean.columns = item_clean.columns.str.strip().str.lower()\
    .str.replace(' ', '').str.replace('(', '').str.replace(')', '')\
    .str.replace('-', '').str.replace('_', '')

item_clean.rename(columns={'itemnumber':'itemno'}, inplace=True)

In [19]:
#clean the item numbers
item_clean['itemno'] = item_clean['itemno'].str.strip()

In [20]:
#print(item_clean['postinggroup'].unique)
item_clean['postinggroup']=item_clean['postinggroup'].str.strip().str.replace(' ', '').str.lower()
print(item_clean['postinggroup'].unique())
#compare to the sales dataset
print(sales_clean['postinggroup'].unique())
#Per emma, while they're not the same changing these bewteen the two sets would require a database administrator so for purposes
#of this exercise we'll leave them as is

['fgseedki' 'raw' 'accessory' 'fggardens' 'seedpod' 'displays' 'seedkits'
 'aerogarden' 'component']
['fgseedki' 'accessory' 'fggardens' 'raw' 'displays' 'seedpod' 'component'
 'ag3garden']


In [21]:
#clean the description data the same way we cleaned it for the sales data
item_clean['description']=item_clean['description'].astype(str).str.strip().str.lower().str.replace('_','')\
.str.replace(' ','').str.replace('-','').str.replace('(','').str.replace(')','')\
.str.replace('$','').str.replace('.','').str.replace('nan','').str.replace("'",'').str.replace('&', '')\
.str.replace('/', '').str.replace("''", "").str.replace('pod', 'p')\
    .str.replace('sk', '').str.replace('acc', '').str.replace('rp', '')\
    .str.replace('pack', 'pk').str.replace('w/', '').str.replace('us', '')\
    .str.replace('eu', '').str.replace('uni', '').str.replace('dlx', '')\
    .str.replace('seedstartingsystem', 'sss').str.replace('seedstartingsys', 'sss')\
    .str.replace('aggrowbowlv2growmedia', 'growbowlandgrowmedia').str.replace('ag', '')\
    .str.replace('display', '').str.replace('seedp', '').str.replace('std', '')\
    .str.replace('pop', '').str.replace('aerogdn', '').str.replace('aerogarden', '')\
    .str.replace('pdq', '').str.replace('versal', '').str.replace('bat', '').str.replace('deluxe', '')\
    .str.replace('wb', '').str.replace('gherb', 'gh').str.replace('gourmetherb', 'gh')

item_clean['description']=item_clean['description'].str.replace(',', ' ').str.strip()
item_clean.head()


,itemno,postinggroup,description
0,0020-00Z,fgseedki,14vps saladlovers
1,0021-00Z,fgseedki,14p herbloversseedkit
2,100235-0000,raw,flatplantspacerchina
3,100824-0000,raw,supergrownutrients3oz
4,100828-0000,accessory,aerovoirstand


In [22]:
#epand the description column to match the sales dataframe
exp_item_clean= item_clean['description'].str.split(' ',n=-1,expand=True)
exp_item_clean.head()

,0,1,2,3,4
0,14vps,saladlovers,None,None,None
1,14p,herbloversseedkit,None,None,None
2,flatplantspacerchina,None,None,None,None
3,supergrownutrients3oz,None,None,None,None
4,aerovoirstand,None,None,None,None


In [23]:
item_clean['desc1'] = exp_item_clean[0]
item_clean['desc2'] = exp_item_clean[1]
item_clean['desc3'] = exp_item_clean[2]
item_clean['desc4'] = exp_item_clean[3]
item_clean = item_clean.drop(columns=["description"])
item_clean['desc1'] = item_clean['desc1'].str.replace('7','7p').str.replace('6','6p')\
.str.replace('3','3p').str.replace('None','').str.replace('7ppp','7p').str.replace('7pp','7p')\
.str.replace('6pp','6p').str.replace('3pp','3p')
item_clean.fillna(value="-",inplace=True)
item_clean.head()

,itemno,postinggroup,desc1,desc2,desc3,desc4
0,0020-00Z,fgseedki,14vps,saladlovers,-,-
1,0021-00Z,fgseedki,14p,herbloversseedkit,-,-
2,100235-0000,raw,flatplantspacerchina,-,-,-
3,100824-0000,raw,supergrownutrients3poz,-,-,-
4,100828-0000,accessory,aerovoirstand,-,-,-


In [24]:
item_clean.to_csv('data/item_clean.csv', index=False) 

In [25]:
## 3) Clean the Customer data file

In [26]:
csv_path = "data/customerlookup_raw.csv"
customer_clean = pd.read_csv(csv_path)
customer_clean.head()

,Sell-to Customer No_,CustomerType
0,3XL001,Retail
1,ACE001,Retail
2,AGOPS,Retail
3,AGSALES,Retail
4,AM3001,Retail


In [27]:
#clean the columns and customer data
customer_clean.columns = customer_clean.columns.str.strip().str.lower()\
    .str.replace(' ', '').str.replace('(', '').str.replace(')', '')\
    .str.replace('-', '').str.replace('_', '')

customer_clean.rename(columns = {'selltocustomerno':'customerno'}, inplace = True)
customer_clean["customerno"] = customer_clean["customerno"].str.strip()

In [28]:
customer_clean.to_csv('data/customer_clean.csv', index=False) 